# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
## import the standard packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


## import hypothesis testing functions
from scipy import stats

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
# set working dataframe
df = pd.read_csv('homepage_actions.csv')

# display info and head
display(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


None

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [9]:
click_ids = set(df[df.action == 'click']['id'].unique())
view_ids = set(df[df.action == 'view']['id'].unique())
print('Viewers who did not click:', len(view_ids-click_ids))
print('Clickers who did not view:', len(click_ids-view_ids))

Viewers who did not click: 4468
Clickers who did not view: 0


In [10]:
exp_ids = set(df[df.group == 'experiment']['id'].unique())
con_ids = set(df[df.group ==  'control']['id'].unique())
print('Users in experiments & control:', len(exp_ids & con_ids))

Users in experiments & control: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [18]:
df['count'] = 1

In [20]:
# create the control dataframe
# slice the df on the control group, then pivot on id. the columns set to
# action will give us a column for click and one for view. The table then
# uses the value from count (1) to put in for that value
control = df[df.group == 'control'].pivot(index='id', columns='action',
                                          values='count')
# change NaNs to 0.0
control = control.fillna(value=0)

In [21]:
# do the same thing for experiment dataframe
experiment = df[df.group == 'experiment'].pivot(index='id', columns='action',
                                                values='count')
experiment = experiment.fillna(value=0)

In [37]:
print('\t\tControl\t\tExperiment')
print(f'Sample sizes:\t {len(control)}\t\t{len(experiment)}')
print(f'Total clicks:\t {control.click.sum()}\t\t{experiment.click.sum()}')
print(f'Click rate:\t {round(control.click.mean(),3)}\t\t{round(experiment.click.mean(),3)}')

		Control		Experiment
Sample sizes:	 3332		2996
Total clicks:	 932.0		928.0
Click rate:	 0.28		0.31


In [42]:
stats.ttest_ind(control, experiment, axis=0)

Ttest_indResult(statistic=array([-2.61956968,         nan]), pvalue=array([0.0088251,       nan]))

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [44]:
#Your code here
control_cr = control.click.mean()
expected_clicks = control_cr * len(experiment)
print(expected_clicks)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [46]:
#Your code here
n = len(experiment)
p = control_cr
var = n * p * (1-p)
std = np.sqrt(var)
print(std)

24.568547907005815


In [47]:
actual_clicks = experiment.click.sum()
z = (actual_clicks - expected_clicks)/std
print(z)

3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [48]:
p = stats.norm.sf(z)
print(p)

0.00012486528006951198


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: Yes, the do.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.